In [ ]:
!pip install -q torch opencv-python pandas
!pip install -q ultralytics matplotlib
!pip install -q huggingface_hub

## Kaggle

In [ ]:
import kagglehub

root_dir = "/kaggle/working"

dataset_dir = kagglehub.dataset_download("ayushspai/sportsmot")

print("Path to dataset files:", dataset_dir)

## Colab

In [ ]:
# !pip install -q kaggle
# !kaggle datasets download -d ayushspai/sportsmot -q
# !unzip -q sportsmot.zip -d SportsMOT
# !rm sportsmot.zip
# !mv /content/SportsMOT/sportsmot_publish/dataset/val/* /content/SportsMOT/sportsmot_publish/dataset/train > /dev/null 2>&1

In [ ]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, ColorJitter

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from ultralytics import YOLO

import shutil

from huggingface_hub import HfApi, Repository, hf_hub_download

import time

## Dataset

In [ ]:
def load_football_sequences(splits_dir):
    football_file = os.path.join(splits_dir, "football.txt")
    with open(football_file, "r") as f:
        football_sequences = f.read().splitlines()
    return set(football_sequences)

def copy_football_data(root_dir, dataset_dir):
    destination_dir = os.path.join(root_dir, "sportsmot")
    os.makedirs(destination_dir, exist_ok=True)
    
    splits_dir = os.path.join(dataset_dir, "sportsmot_publish", "splits_txt")
    football_sequences = load_football_sequences(splits_dir)
    shutil.copytree(splits_dir, os.path.join(destination_dir, "splits_txt"), dirs_exist_ok=True)
    
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(dataset_dir, "sportsmot_publish", "dataset", split)
        dest_split_dir = os.path.join(destination_dir, split)
        os.makedirs(dest_split_dir, exist_ok=True)
        
        if not os.path.exists(split_dir):
            continue

        sequences = os.listdir(split_dir)
        for seq in sequences:
            seq_path = os.path.join(split_dir, seq)
            dest_seq_path = os.path.join(dest_split_dir, seq)
            
            if seq in football_sequences:
                shutil.copytree(seq_path, dest_seq_path, dirs_exist_ok=True)

    print("Football data copying complete!")
    return destination_dir

dataset_dir = copy_football_data(root_dir, dataset_dir)
print("Path to dataset files:", dataset_dir)

## Object Detection

### Converting Dataset

In [ ]:
def convert_to_yolo_format(root_dir, dataset_dir, split):
    os.makedirs(os.path.join(root_dir, "dataset", "images", split), exist_ok=True)
    os.makedirs(os.path.join(root_dir, "dataset", "labels", split), exist_ok=True)

    splits_dir = os.path.join(dataset_dir, "splits_txt")
    football_sequences = load_football_sequences(splits_dir)

    for seq in football_sequences:
        seq_path = os.path.join(dataset_dir, split, seq)
        gt_file = os.path.join(seq_path, "gt", "gt.txt")

        if not os.path.exists(gt_file):
            continue

        df = pd.read_csv(gt_file, header=None)
        df.columns = [
            "frame_id", "object_id", "x", "y", "w", "h",
            "conf", "class_id", "visibility"
        ]

        for frame_id, group in df.groupby("frame_id"):
            img_path = os.path.join(seq_path, "img1", f"{frame_id:06d}.jpg")
            if not os.path.exists(img_path):
                continue

            img = cv2.imread(img_path)
            img_name = f"{seq}_{frame_id:06d}.jpg"
            cv2.imwrite(os.path.join(root_dir, "dataset", "images", split, img_name), img)

            label_file = os.path.join(root_dir, "dataset", "labels", split, f"{seq}_{frame_id:06d}.txt")
            with open(label_file, "w") as f:
                for _, row in group.iterrows():
                    x_center = (row["x"] + row["w"] / 2) / img.shape[1]
                    y_center = (row["y"] + row["h"] / 2) / img.shape[0]
                    width = row["w"] / img.shape[1]
                    height = row["h"] / img.shape[0]
                    f.write(f"{int(row['class_id'])} {x_center} {y_center} {width} {height}\n")

convert_to_yolo_format(root_dir, dataset_dir, "train")
convert_to_yolo_format(root_dir, dataset_dir, "val")
convert_to_yolo_format(root_dir, dataset_dir, "test")

### YOLO

In [ ]:
def create_yaml_file(dataset_dir, class_names):
    yaml_content = f"""
path: {dataset_dir}
train: images/train
val: images/val
test: images/test

nc: {len(class_names)}
names: {class_names}
"""
    yaml_path = os.path.join(dataset_dir, "dataset.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print(f"YAML file created at: {yaml_path}")

yolo_dataset_dir = "/kaggle/working/dataset"
class_names = ['player', 'referee', 'ball', 'other'] # TODO

create_yaml_file(yolo_dataset_dir, class_names)

In [ ]:
model = YOLO("yolov8n.pt")

results = model.train(
    data=f"{yolo_dataset_dir}/dataset.yaml",
    epochs=50,
    imgsz=1280,
    batch=16,
    name="yolov8_football",
    verbose=False
)

### Logs

In [ ]:
def plot_logs(results_path):
    logs = pd.read_csv(results_path)

    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["train/box_loss"], label="Train Box Loss")
    plt.plot(logs["epoch"], logs["val/box_loss"], label="Validation Box Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["metrics/mAP_0.5"], label="mAP@0.5")
    plt.plot(logs["epoch"], logs["metrics/mAP_0.5:0.95"], label="mAP@0.5:0.95")
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title("Mean Average Precision")
    plt.legend()
    plt.show()

results_path = "/kaggle/working/runs/detect/yolov8_football/results.csv"
plot_logs(results_path)

### Testing and Validation

In [ ]:
model_path = "/kaggle/working/runs/detect/yolov8_football/weights/best.pt"

model = YOLO(model_path)

results = model("/kaggle/working/dataset/images/test/seq101_000001.jpg") # TODO

for result in results:
    result.show()
    result.save("output.jpg")

In [ ]:
metrics = model.val()

print(f"mAP@0.5: {metrics.box.map}")
print(f"mAP@0.5:0.95: {metrics.box.map_50_95}")

## Saving to Hugging Face


In [ ]:
!huggingface-cli login

In [ ]:
model_path = "/content/runs/detect/yolov8_football/weights/best.pt"
repo_id = "ParsaGh/yolov8_football"
repo_name = "yolov8_football"

model = YOLO(model_path)

repo = Repository(l
    ocal_dir=yolov8_football,
    clone_from=repo_id,
    use_auth_token=True
)
os.system(f"cp {model_path} {repo_name}/")
repo.push_to_hub(commit_message="Upload YOLOv8-Football model")

## Loading From Hugging Face

In [ ]:
model_path = hf_hub_download(
    repo_id=repo_id,
    filename="best.pt",
    use_auth_token=True,
)

model = YOLO(model_path)

# results = model("path_to_test_image.jpg") #TODO
# results.show()

## Single Object Tracking

In [ ]:
class SingleObjectTracker:
    def __init__(self, yolo_weights='best.pt'):
        self.yolo = YOLO(yolo_weights)
        self.tracker = cv2.TrackerCSRT_create()
        self.target_class = 0
        self.last_valid_bbox = None
        self.frames_since_last_detection = 0
        self.heatmap = None
        self.heatmap_alpha = 0.5

    def track(self, frame):
        success, bbox = self.tracker.update(frame)

        if not success or self._needs_reinitialization(bbox):
            print("Attempting recovery...")
            success, bbox = self._reinitialize_tracker(frame)

        if success:
            self.last_valid_bbox = bbox
            self.frames_since_last_detection = 0
        else:
            self.frames_since_last_detection += 1

        return success, bbox

    def _needs_reinitialization(self, bbox):
        if bbox[2] <= 0 or bbox[3] <= 0:
            return True
        if self.last_valid_bbox:
            # Check position change > 50% of frame size
            dx = abs(bbox[0] - self.last_valid_bbox[0])/1280
            dy = abs(bbox[1] - self.last_valid_bbox[1])/720
            return dx > 0.5 or dy > 0.5
        return False

    def _reinitialize_tracker(self, frame):
        # Search in last known position area
        if self.last_valid_bbox:
            x, y, w, h = self.last_valid_bbox
            search_region = frame[int(y):int(y+h), int(x):int(x+w)]
            results = self.yolo(search_region, verbose=False)[0]
        else:
            results = self.yolo(frame, verbose=False)[0]

        # Filter and select best detection
        athlete_boxes = []
        for box, cls, conf in zip(results.boxes.xywh, results.boxes.cls, results.boxes.conf):
            if cls == self.target_class and conf > 0.5:
                athlete_boxes.append((conf, box.cpu().numpy()))

        if athlete_boxes:
            # Select closest to last position or highest confidence
            best_conf, best_box = max(athlete_boxes, key=lambda x: x[0])
            x_center, y_center, w, h = best_box
            x = x_center - w/2 + (self.last_valid_bbox[0] if self.last_valid_bbox else 0)
            y = y_center - h/2 + (self.last_valid_bbox[1] if self.last_valid_bbox else 0)
            new_bbox = (x, y, w, h)
            self.tracker = cv2.TrackerCSRT_create()
            self.tracker.init(frame, new_bbox)
            return True, new_bbox

        return False, None

    def update_heatmap(self, bbox):
        if bbox is None:
            return
        
        # Create heatmap if not initialized
        if self.heatmap is None:
            self.heatmap = np.zeros((720, 1280), dtype=np.float32)
        
        # Extract object center
        x, y, w, h = bbox
        center_x = int(x + w/2)
        center_y = int(y + h/2)
        
        # Add Gaussian blur to the center point
        cv2.circle(self.heatmap, (center_x, center_y), 10, 1, -1)
        self.heatmap = cv2.GaussianBlur(self.heatmap, (51, 51), 0)

    def draw_heatmap(self, frame):
        if self.heatmap is None:
            return frame
        
        # Normalize heatmap to 0-255
        heatmap_norm = cv2.normalize(self.heatmap, None, 0, 255, cv2.NORM_MINMAX)
        heatmap_norm = np.uint8(heatmap_norm)
        
        # Apply color map (Jet for better visualization)
        heatmap_colored = cv2.applyColorMap(heatmap_norm, cv2.COLORMAP_JET)
        
        # Overlay heatmap on the frame
        overlay = cv2.addWeighted(frame, 1 - self.heatmap_alpha, heatmap_colored, self.heatmap_alpha, 0)
        return overlay

### Test

In [ ]:
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Output Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
def test_on_sportsmot_frames(frames_dir, output_dir="results"):
    tracker = SingleObjectTracker('best.pt')
    
    os.makedirs(output_dir, exist_ok=True)
    
    frame_paths = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir)])
    
    frame = cv2.imread(frame_paths[0])
    if frame is None:
        raise ValueError("No frames found in test directory")
    
    height, width = frame.shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join(output_dir, 'tracking_results.mp4'), 
                         fourcc, 25.0, (width, height))
    
    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        if frame is None:
            continue
            
        if tracker.last_valid_bbox is None:
            bbox = tracker.initialize(frame)
        else:
            success, bbox = tracker.track(frame)
        
        if bbox is not None:
            tracker.update_heatmap(bbox)
        
        frame_with_heatmap = tracker.draw_heatmap(frame)
        
        if bbox is not None:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame_with_heatmap, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        out.write(frame_with_heatmap)
    
    out.release()
    print(f"Results saved to: {os.path.abspath(output_dir)}")
    play_video(os.path.join(output_dir, 'tracking_results.mp4')

In [ ]:
sportsmot_test_dir = f"{dataset_dir}/test/" # TODO
test_on_sportsmot_frames(sportsmot_test_dir)

### Conditions Where CSRT Performs Well:


Slow Motion: The object moves slowly relative to the frame rate.

Minimal Occlusion: The object is not occluded by other objects.

Consistent Lighting: The lighting conditions remain stable.

Distinct Appearance: The object has a unique appearance compared to the background.

### Conditions Where CSRT Performs Poorly:

Fast Motion: The object moves quickly, causing the tracker to lose it.

Occlusion: The object is occluded by other objects or goes out of the frame.

Appearance Changes: The object's appearance changes significantly (e.g., due to lighting or deformation).

Similar Background: The object blends into the background, making it hard to distinguish.

## Multiple Object Tracking